# Introduction to RAG (Retrieval-Augmented Generation)

**This notebook provides a reference solution using LangChain.**

## 📚 Learning Objectives

By the end of this tutorial, you will be able to:
- Understand the core concepts of RAG (Retrieval-Augmented Generation)
- Implement a basic RAG pipeline using LangChain
- Create vector embeddings and perform semantic search
- Build a question-answering system with context retrieval
- Visualize and analyze the RAG pipeline components

## 🎯 What is RAG?

**Retrieval-Augmented Generation (RAG)** is a technique that combines:
1. **Retrieval**: Finding relevant information from a knowledge base
2. **Augmentation**: Injecting this context into the prompt
3. **Generation**: Using a language model to generate answers based on the retrieved context

This approach improves factual accuracy and reduces hallucinations by grounding the model's responses in actual documents.


In [ ]:
# Compatibility fix for langchain_google_genai with newer langchain_core
# Run this cell FIRST before importing langchain_google_genai
import sys
import pydantic
import warnings

# Create a compatibility shim for pydantic_v1
# This provides Pydantic v1 compatibility for langchain_google_genai
class PydanticV1Compat:
    """Compatibility shim for pydantic_v1 imports"""
    def __getattr__(self, name):
        # Handle root_validator specially for Pydantic v2 compatibility
        if name == 'root_validator':
            # Return a decorator that works with Pydantic v2
            def root_validator(*args, **kwargs):
                # Suppress warnings for deprecated root_validator
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    # Try to use model_validator if available (Pydantic v2)
                    if hasattr(pydantic, 'model_validator'):
                        return pydantic.model_validator(mode='before', *args, **kwargs)
                    # Fallback to field_validator or other v2 validators
                    return lambda f: f
            return root_validator
        return getattr(pydantic, name)

# Patch langchain_core.pydantic_v1 if it doesn't exist
try:
    import langchain_core
    if not hasattr(langchain_core, 'pydantic_v1'):
        langchain_core.pydantic_v1 = PydanticV1Compat()
        sys.modules['langchain_core.pydantic_v1'] = langchain_core.pydantic_v1
        print("✅ Compatibility fix applied for langchain_google_genai")
except Exception as e:
    print(f"⚠️ Warning: Could not apply compatibility fix: {e}")



## 📦 Installation

First, let's install the required libraries:
- `langchain`: Framework for building LLM applications
- `langchain-openai`: OpenAI integration for LangChain
- `chromadb`: Vector database (alternative to Qdrant)
- `qdrant-cpu`: Facebook AI Similarity Search for efficient vector similarity search
- `tiktoken`: Tokenizer for OpenAI models
- `pypdf`: PDF processing library (for future exercises)
- `matplotlib`, `seaborn`: For data visualization
- `numpy`: For numerical operations
- `scikit-learn`: For PCA and similarity calculations


In [ ]:
import langchain
import langchain_community
print(langchain.__version__)
print(langchain_community.__version__)
!uv pip freeze | grep qdrant

## 🔐 API Key Setup

Make sure you have your OpenAI API key set in your environment variables. You can set it by:
- Exporting it in your terminal: `export OPENAI_API_KEY="your-key-here"`
- Or using a `.env` file with `python-dotenv`


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

# Determine which backend to use in the .env
USE_OLLAMA = os.environ.get("USE_OLLAMA", "").lower() in ("1", "true", "yes")
USE_GEMINI = os.environ.get("USE_GEMINI", "").lower() in ("1", "true", "yes")

if USE_OLLAMA:
    # Utiliser Ollama
    from langchain_community.chat_models import ChatOllama
    # Assurez-vous que le serveur Ollama est en cours d'exécution
    # et que le modèle 'mistral' est téléchargé.
    llm = ChatOllama(model="mistral", temperature=0)
    print("⚙️ Utilisation du backend Ollama (mistral)")
elif USE_GEMINI:
    # Utiliser Gemini
    # Assurez-vous que la variable d'environnement GOOGLE_API_KEY est définie dans le .env
    from langchain_google_genai import ChatGoogleGenerativeAI
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
    print("⚙️ Utilisation du backend Gemini (gemini-2.5-flash)")
else:
    # Utiliser OpenAI par défaut
    # Assurez-vous que la variable d'environnement OPENAI_API_KEY est définie dans le .env
    from langchain_openai.chat_models import ChatOpenAI
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    print("⚙️ Utilisation du backend OpenAI (gpt-4o-mini)")

## 🚀 Building a Basic RAG Pipeline

Let's build a complete RAG pipeline step by step. The pipeline consists of:

1. **Document Loading**: Prepare your documents
2. **Text Splitting**: Break documents into smaller chunks
3. **Embedding**: Convert text chunks into vector embeddings
4. **Vector Storage**: Store embeddings in a vector database
5. **Retrieval**: Find relevant chunks for a query
6. **Generation**: Generate answers using retrieved context

Let's implement this:


In [ ]:
# ✅ Modern LangChain imports (compatible with LangChain 0.1+)
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

# Step 1: Define our sample documents
documents_raw = [
    """Mr. Jonas, a fluffy orange cat, decided he was tired of just napping all day. He applied for a promotion at the local office. Unfortunately, during the interview, he fell asleep on the keyboard, accidentally sending a company-wide email that said, 'Meow if you approve!' Surprisingly, the staff loved it, thinking it was a new motivational strategy. Mr. Jonas got the promotion, now supervising the office naps. Soon, he implemented 'cat breaks' every hour, where employees were encouraged to stretch, purr softly, or chase imaginary laser pointers. Productivity oddly skyrocketed, and the office became famous for its feline-inspired efficiency seminars. Mr. Jonas even started a newsletter titled 'The Purr-fect Manager,' sharing tips on balancing work and naps.""",
    """Jenny bought a toaster from a mysterious yard sale, not realizing it was a time machine. Every time she toasted bread, it popped up with tiny messages from the future. One morning, the toast said, 'Beware the dancing squirrels at 3 PM!' She ignored it, and sure enough, a flash mob of squirrels waltzed through her garden, all wearing tiny top hats. Jenny now always eats her breakfast with a helmet. Eventually, she discovered a secret lever inside the toaster that allowed her to send messages back in time. She used it to warn her past self about spilled coffee, missed deadlines, and unexpected squirrel invasions. Breakfast became an adventure, with toast revealing mysteries of the universe and sometimes giving advice on what socks to wear for luck.""",
    """In the small town of Noodleton, spaghetti rained from the sky for no reason at all. Mayor Luigi tried to stop it with umbrellas, but the noodles were relentless. One ambitious dog named Max decided to chase the falling spaghetti like a game. Soon, the whole town joined, slipping, sliding, and twirling in a chaotic noodle dance-off. By evening, everyone agreed: it was the tastiest disaster in history. Over the next few days, chefs experimented with sky-spaghetti recipes, inventing noodle ice cream, spaghetti smoothies, and pasta hats. Scientists tried to figure out the phenomenon but concluded it was 'spaghetti magic.' Max became the town hero, leading annual 'noodle festivals,' and Noodleton transformed into the world's first pasta-themed tourist destination."""
]

# Step 2: Convert raw texts to Document objects
docs = [Document(page_content=txt, metadata={"source": "local", "idx": i})
        for i, txt in enumerate(documents_raw)]

print(f"✅ Created {len(docs)} documents")
for i, doc in enumerate(docs):
    print(f"\nDocument {i}:")
    print(f"  Content: {doc.page_content[:100]}...")
    print(f"  Metadata: {doc.metadata}")

### 📏 Text Splitting (Chunking)

**Why chunk documents?**
- Language models have token limits (context windows)
- Smaller chunks are easier to retrieve precisely
- Overlapping chunks preserve context at boundaries

**Parameters:**
- `chunk_size`: Maximum size of each chunk (in characters)
- `chunk_overlap`: Overlap between chunks to preserve context


In [ ]:
# Step 3: Split documents into chunks
# RecursiveCharacterTextSplitter tries to split on paragraphs, sentences, then words
# to preserve semantic meaning as much as possible
CHUNK_SIZE = 200

splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,      # Maximum characters per chunk
    chunk_overlap=50     # Overlap between chunks (helps preserve context)
)
chunks = splitter.split_documents(docs)

print(f"Split {len(docs)} documents into {len(chunks)} chunks")
print("\nChunk examples:")
for i, chunk in enumerate(chunks[:3]):
    print(f"\n--- Chunk {i} ---")
    print(f"Length: {len(chunk.page_content)} characters")
    print(f"Content: {chunk.page_content}")
    print(f"Metadata: {chunk.metadata}")

#### What happens if `chunk_size` is too small? Too large?

- **If `chunk_size` is too small:**  
  - **Pros:** Each chunk is very focused, minimizing irrelevant information.  
  - **Cons:** Important context can be lost because sentences/paragraphs are split. Retrieval may return partial or incoherent answers.  
  - **Performance:** Number of chunks increases → more memory/storage usage and slower embedding/search.

- **If `chunk_size` is too large:**  
  - **Pros:** More context is preserved in each chunk, helping the model produce complete answers.  
  - **Cons:** Irrelevant text may be included, which can confuse retrieval.  
  - **Performance:** Fewer chunks → faster embedding, but long chunks may exceed the model’s token limit during retrieval/generation.

**Rule of thumb:** Choose chunks that fit comfortably within your LLM’s context window (e.g., 500–1000 tokens for GPT-3.5/GPT-4 small tasks).

#### How does `chunk_overlap` affect retrieval quality?

- `chunk_overlap` defines how many tokens/characters are shared between consecutive chunks.  

- **Higher overlap:**  
  - **Pros:** Preserves context across chunk boundaries → improves retrieval for multi-sentence queries.  
  - **Cons:** More redundant chunks → slightly higher storage and embedding costs.  

- **Lower overlap:**  
  - **Pros:** Less redundancy → faster processing and lower storage.  
  - **Cons:** May lose context at chunk boundaries → retrieval can miss important information.  

**Tip:** Typical overlap is 50–100 tokens (~20–30% of chunk size).

#### What's the optimal chunk size for your use case?

- **Depends on:**  
  - Document type (dense technical PDFs vs. short news articles)  
  - Query complexity (simple fact lookup vs. multi-sentence reasoning)  
  - LLM context window (e.g., GPT-4 has 8k–32k tokens)  

**Practical guideline:**  
- Start with:  
  - `chunk_size = 500–1000 tokens`  
  - `chunk_overlap = 50–100 tokens`  

- Test retrieval quality:  
  - If answers are incomplete → increase chunk size or overlap.  
  - If answers include too much irrelevant info → decrease chunk size.  

💡 **Extra tip:** Visualize a few chunks to ensure they contain coherent, self-contained information. This is often the easiest way to find the “sweet spot.”


### Minimal RAG Implementation (Without LangChain Wrappers)

Below is a compact reference implementation showing how to:

- Call the embeddings API directly (supports OpenAI and Gemini based on `USE_GEMINI` environment variable)
- Build an in-memory index
- Perform cosine similarity search
- Generate answers using the configured LLM backend (OpenAI or Gemini)

This helps you understand what happens under the hood of LangChain. The implementation automatically uses the backend specified by your environment variables (same as the rest of the notebook).

In [ ]:
import os
import numpy as np
from dotenv import load_dotenv

load_dotenv()

# Determine which backend to use
USE_OLLAMA = os.environ.get("USE_OLLAMA", "").lower() in ("1", "true", "yes")
USE_GEMINI = os.environ.get("USE_GEMINI", "").lower() in ("1", "true", "yes")

def embed_texts(texts):
    """Convert texts to embeddings using the configured backend"""
    if USE_GEMINI:
        # Use Google Gemini embeddings
        import google.generativeai as genai
        genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))
        # Embed multiple texts
        embeddings_list = []
        for text in texts:
            result = genai.embed_content(
                model="models/embedding-001",
                content=text,
                task_type="retrieval_document"
            )
            embeddings_list.append(result['embedding'])
        return embeddings_list
    else:
        # Use OpenAI embeddings (default)
        from openai import OpenAI
        client = OpenAI()
        resp = client.embeddings.create(
            model="text-embedding-3-small", 
            input=texts
        )
        return [d.embedding for d in resp.data]

def generate_answer(context, query):
    """Generate answer using the configured LLM backend"""
    if USE_GEMINI:
        # Use Google Gemini
        import google.generativeai as genai
        genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))
        model = genai.GenerativeModel("gemini-2.5-flash")
        
        prompt = f"""You are a helpful assistant. Use the provided context chunks to comprehensively answer the user's query. Do not use external knowledge. If the context does not contain enough information, state that clearly.

{context}

User Query: {query}"""
        
        response = model.generate_content(prompt)
        return response.text
    else:
        # Use OpenAI (default)
        from openai import OpenAI
        client = OpenAI()
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant. Use the provided context chunks to comprehensively answer the user's query. Do not use external knowledge. If the context does not contain enough information, state that clearly."},
                {"role": "user", "content": f"{context}\n\nUser Query: {query}"}
            ]
        )
        return completion.choices[0].message.content

# Index in memory
raw_texts = [c.page_content for c in chunks]  # Reuse 'chunks' from above
vecs = embed_texts(raw_texts)

def cos_sim(a, b):
    """Calculate cosine similarity between two vectors"""
    return float(np.dot(a, b) / ((np.linalg.norm(a) + 1e-8) * (np.linalg.norm(b) + 1e-8)))

def search(query, k=3):
    """Search for k most similar chunks to the query"""
    # Get query embedding
    qv = embed_texts([query])[0]
    
    # Calculate similarities
    scored = [(cos_sim(qv, v), t) for v, t in zip(vecs, raw_texts)]
    
    # Sort by similarity (descending)
    scored.sort(key=lambda x: x[0], reverse=True)
    
    # Return top k
    return [(score, text) for score, text in scored[:k]]

# Quick demo
results = search("Explain Jonas's story", k=2)
for score, text in results:
    print(f"Similarity: {score:.3f}")
    print(f"Text: {text[:100]}...")
    print()

# Prepare context for the LLM
context = ""
for score, text in results:
    context += f"***Similarity Score: {score:.3f}***\nCONTEXT CHUNK:\n{text}\n---\n"

# Final LLM call to generate a complete answer based on the retrieved context
final_answer = generate_answer(context, "Explain Jonas's story")

# Print the final generated answer
print("\n--- FINAL ANSWER GENERATED BY LLM ---\n")
print(final_answer)

### 🔢 Creating Embeddings and Vector Store

**Embeddings** are numerical representations of text that capture semantic meaning.
Similar texts have similar embeddings, allowing us to find relevant content through
vector similarity search.

**Qdrant** is a library for efficient similarity search and clustering of dense vectors.

In [ ]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

# Step 4: Create embeddings based on backend selection
# Check which backend to use (from the setup cell)
USE_OLLAMA = os.environ.get("USE_OLLAMA", "").lower() in ("1", "true", "yes")
USE_GEMINI = os.environ.get("USE_GEMINI", "").lower() in ("1", "true", "yes")

if USE_GEMINI:
    # Use Google Gemini embeddings
    from langchain_google_genai import GoogleGenerativeAIEmbeddings
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    embedding_dim = 768  # Gemini embedding dimension
    print("⚙️ Using Google Gemini embeddings")
else:
    # Use OpenAI embeddings (default)
    from langchain_openai import OpenAIEmbeddings
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    embedding_dim = 1536  # OpenAI text-embedding-3-small dimension
    print("⚙️ Using OpenAI embeddings")

embeddings

In [ ]:
# Step 5: Create vector store from chunks
# Qdrant stores the embeddings and enables fast similarity search
# Use location=":memory:" for in-memory storage

# Create Qdrant client
client = QdrantClient(location=":memory:")

# Delete collection if it exists to start fresh
try:
    client.delete_collection('docs')
except Exception:
    pass

# Create collection with proper configuration
from qdrant_client.models import Distance, VectorParams
client.create_collection(
    collection_name='docs',
    vectors_config=VectorParams(
        size=embedding_dim,
        distance=Distance.COSINE
    )
)

# Create vector store using QdrantVectorStore (recommended, non-deprecated)
vectordb = QdrantVectorStore(
    client=client,
    collection_name='docs',
    embedding=embeddings
)

# Add documents to the vector store
vectordb.add_documents(chunks)

# Get the number of documents (Qdrant doesn't support len() directly)
# We can check by getting all documents or using the client
print(f"✅ Vector store created successfully")
print(f"   Number of chunks: {len(chunks)}")
print(f"   Embedding dimension: {embedding_dim}")

### 🔍 Setting Up the Retriever and QA Chain

A **retriever** finds the most relevant chunks for a query.
The **QA chain** combines retrieval with generation to answer questions.


In [ ]:
# Step 6: Create a retriever
# The retriever will find the k most similar chunks for each query
retriever = vectordb.as_retriever(search_kwargs={"k": 3})  # Retrieve top 3 chunks

# Step 7: Create the QA chain using modern LangChain patterns
# Use the llm from the setup cell (already configured for Gemini/OpenAI/Ollama)

# Create a prompt template for the QA chain
system_prompt = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Keep the answer concise and focused on the information provided in the context.

{context}"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

# Create the document chain (handles the "stuff" approach - concatenates all docs)
document_chain = create_stuff_documents_chain(llm, prompt)

# Create the retrieval chain (combines retriever + document chain)
qa = create_retrieval_chain(retriever, document_chain)

print("✅ RAG pipeline ready!")

### 🎯 Testing the RAG System

Let's ask a question and see how the system retrieves relevant context and generates an answer.


In [ ]:
# Ask a question
questions = [
    "Who is Mr. Jonas?",
    "What Jenny's toast said?",
    "Which object the Mayor Luigi used to try to stop the Spaghetti rain?",
]
for question in questions:
    answer = qa.invoke({"input": question})
    
    print("=" * 60)
    print("QUESTION:")
    print(question)
    print("\n" + "=" * 60)
    print("ANSWER:")
    print(answer["answer"])
    print("=" * 60)

## 📊 Visualizing the RAG Pipeline

Let's create some visualizations to better understand what's happening in the RAG pipeline.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

# Set style for better-looking plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

### 📈 Visualization 1: Document Chunking Analysis

Let's visualize how documents are split into chunks and their sizes.


In [ ]:
# Visualize chunk sizes
chunk_sizes = [len(chunk.page_content) for chunk in chunks]
chunk_indices = list(range(len(chunks)))

fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart of chunk sizes
ax[0].bar(chunk_indices, chunk_sizes, color='skyblue', edgecolor='navy', alpha=0.7)
ax[0].axhline(y=CHUNK_SIZE, color='r', linestyle='--', label='Max chunk size (CHUNK_SIZE)')
ax[0].set_xlabel('Chunk Index')
ax[0].set_ylabel('Chunk Size (characters)')
ax[0].set_title('Chunk Sizes Distribution')
ax[0].legend()
ax[0].grid(True, alpha=0.3)

# Box plot
ax[1].boxplot(chunk_sizes, vert=True, patch_artist=True,
              boxprops=dict(facecolor='lightblue', alpha=0.7))
ax[1].set_ylabel('Chunk Size (characters)')
ax[1].set_title('Chunk Size Statistics')
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Statistics:")
print(f"  Total chunks: {len(chunks)}")
print(f"  Average size: {np.mean(chunk_sizes):.1f} characters")
print(f"  Min size: {min(chunk_sizes)} characters")
print(f"  Max size: {max(chunk_sizes)} characters")
print(f"  Std deviation: {np.std(chunk_sizes):.1f} characters")

### 🔍 Visualization 2: Embedding Similarity Matrix

Let's visualize the similarity between different chunks using their embeddings.
This helps us understand how semantically related our documents are.


In [ ]:
# Get embeddings for all chunks
chunk_texts = [chunk.page_content for chunk in chunks]
chunk_embeddings = embeddings.embed_documents(chunk_texts)

# Calculate cosine similarity matrix
similarity_matrix = cosine_similarity(chunk_embeddings)

# Create heatmap
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(similarity_matrix, cmap='YlOrRd', aspect='auto', vmin=0, vmax=1)

# Add colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Cosine Similarity', rotation=270, labelpad=20)

# Set labels
ax.set_xticks(range(len(chunks)))
ax.set_yticks(range(len(chunks)))
ax.set_xticklabels([f'Chunk {i}' for i in range(len(chunks))], rotation=45, ha='right')
ax.set_yticklabels([f'Chunk {i}' for i in range(len(chunks))])
ax.set_title('Cosine Similarity Matrix Between Chunks', fontsize=14, pad=20)

# Add text annotations
for i in range(len(chunks)):
    for j in range(len(chunks)):
        text = ax.text(j, i, f'{similarity_matrix[i, j]:.2f}',
                      ha="center", va="center", color="black", fontsize=8)

plt.tight_layout()
plt.show()

# Print most similar chunk pairs
print("\nMost similar chunk pairs:")
for i in range(len(chunks)):
    for j in range(i+1, len(chunks)):
        sim = similarity_matrix[i, j]
        if sim > 0.7:  # Threshold for high similarity
            print(f"  Chunk {i} <-> Chunk {j}: {sim:.3f}")

### 🎯 Visualization 3: Query-Chunk Similarity

Let's see which chunks are retrieved for different queries and their similarity scores.


In [ ]:
# Test multiple queries
test_queries = [
    "Who is Mr. Jonas?",
    "What Jenny's toast said?",
    "Which object the Mayor Luigi used to try to stop the Spaghetti rain?"
]

fig, axes = plt.subplots(len(test_queries), 1, figsize=(12, 4 * len(test_queries)))

for idx, query in enumerate(test_queries):
    # Get query embedding
    query_embedding = embeddings.embed_query(query)
    
    # Calculate similarities with all chunks
    similarities = [cosine_similarity([query_embedding], [emb])[0][0] 
                   for emb in chunk_embeddings]
    
    # Get top k chunks
    top_k_indices = np.argsort(similarities)[-3:][::-1]
    top_k_similarities = [similarities[i] for i in top_k_indices]
    
    # Plot
    ax = axes[idx]
    colors = ['green' if i in top_k_indices else 'lightgray' for i in range(len(chunks))]
    bars = ax.barh(range(len(chunks)), similarities, color=colors, alpha=0.7)
    ax.set_yticks(range(len(chunks)))
    ax.set_yticklabels([f'Chunk {i}' for i in range(len(chunks))])
    ax.set_xlabel('Cosine Similarity Score')
    ax.set_title(f'Query: "{query}"\n(Retrieved chunks in green)', fontsize=12)
    ax.set_xlim(0, 1)
    ax.grid(True, alpha=0.3, axis='x')
    
    # Add similarity values on bars
    for i, (bar, sim) in enumerate(zip(bars, similarities)):
        if i in top_k_indices:
            ax.text(sim + 0.02, i, f'{sim:.3f}', va='center', fontweight='bold')
        else:
            ax.text(sim + 0.02, i, f'{sim:.3f}', va='center', color='gray')

plt.tight_layout()
plt.show()

# Print detailed retrieval results
for query in test_queries:
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print(f"{'='*60}")
    
    # Retrieve using the retriever
    retrieved_docs = retriever.invoke(query)
    
    for i, doc in enumerate(retrieved_docs):
        print(f"\nRetrieved Chunk {i+1} (similarity score shown in plot above):")
        print(f"  Content: {doc.page_content[:150]}...")
        print(f"  Metadata: {doc.metadata}")

### 🗺️ Visualization 4: 2D Embedding Projection

Let's project the high-dimensional embeddings into 2D space using PCA to visualize
the relationships between chunks and queries.


In [ ]:
# Project embeddings to 2D using PCA
pca = PCA(n_components=2, random_state=42)
embeddings_2d = pca.fit_transform(chunk_embeddings)

# Create a query and get its embedding
query = "Who is Mr. Jonas?"
query_embedding = embeddings.embed_query(query)
query_2d = pca.transform([query_embedding])[0]

# Plot
fig, ax = plt.subplots(figsize=(12, 8))

# Plot chunk embeddings
scatter = ax.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], 
                    s=200, c=range(len(chunks)), cmap='viridis', 
                    alpha=0.7, edgecolors='black', linewidth=2)

# Add labels for chunks
for i, (x, y) in enumerate(embeddings_2d):
    ax.annotate(f'Chunk {i}', (x, y), xytext=(5, 5), 
               textcoords='offset points', fontsize=10, fontweight='bold')

# Plot query embedding
ax.scatter(query_2d[0], query_2d[1], s=300, c='red', marker='*', 
          edgecolors='darkred', linewidth=2, label='Query', zorder=5)
ax.annotate('Query', (query_2d[0], query_2d[1]), xytext=(10, 10),
           textcoords='offset points', fontsize=12, fontweight='bold', color='red')

# Draw lines from query to top 3 chunks
retrieved_docs = retriever.invoke(query)
for doc in retrieved_docs:
    # Find chunk index
    chunk_idx = next(i for i, chunk in enumerate(chunks) 
                    if chunk.page_content == doc.page_content)
    ax.plot([query_2d[0], embeddings_2d[chunk_idx, 0]],
           [query_2d[1], embeddings_2d[chunk_idx, 1]],
           'r--', alpha=0.3, linewidth=2)

ax.set_xlabel(f'PC1 (explains {pca.explained_variance_ratio_[0]:.1%} of variance)')
ax.set_ylabel(f'PC2 (explains {pca.explained_variance_ratio_[1]:.1%} of variance)')
ax.set_title('2D PCA Projection of Chunk Embeddings\n(Red lines connect query to retrieved chunks)', 
            fontsize=14, pad=20)
ax.legend()
ax.grid(True, alpha=0.3)

plt.colorbar(scatter, ax=ax, label='Chunk Index')
plt.tight_layout()
plt.show()

print(f"PCA explained variance: {pca.explained_variance_ratio_.sum():.2%}")
print("(Note: 2D projection may not capture all relationships in high-dimensional space)")

### Evaluate RAG Performance

**Task**: 
1. Create a set of test questions and expected answers
2. Implement basic evaluation metrics:
   - Retrieval accuracy (are the right chunks retrieved?)
   - Answer quality (does the answer contain correct information?)
3. Visualize the evaluation results

**Questions to answer**:
- How do you measure RAG system performance?
- What metrics are most important for your use case?
- How can you improve the system based on evaluation results?


In [ ]:
# New test dataset based on the whimsical documents
test_cases = [
    {
        "question": "Who got promoted in the office?",
        "expected_keywords": ["Mr. Jonas", "promotion", "supervising"],
        "expected_chunk_keywords": ["Mr. Jonas", "promotion", "office naps"],
        "category": "character"
    },
    {
        "question": "What unusual feature does Jenny's toaster have?",
        "expected_keywords": ["time machine", "messages from the future", "lever"],
        "expected_chunk_keywords": ["toaster", "time machine", "messages"],
        "category": "object"
    },
    {
        "question": "What fell from the sky in Noodleton?",
        "expected_keywords": ["spaghetti", "rain", "noodle"],
        "expected_chunk_keywords": ["spaghetti", "rained", "town of Noodleton"],
        "category": "event"
    },
    {
        "question": "What events involve animals in the stories?",
        "expected_keywords": ["cat breaks", "squirrels waltzing", "Max chasing spaghetti"],
        "expected_chunk_keywords": ["Mr. Jonas", "squirrels", "Max"],
        "category": "animal"
    },
    {
        "question": "What unusual food creations happened in Noodleton?",
        "expected_keywords": ["noodle ice cream", "spaghetti smoothies", "pasta hats"],
        "expected_chunk_keywords": ["chefs", "sky-spaghetti recipes", "pasta"],
        "category": "food"
    }
]

# Use the extended QA chain if available
qa_for_eval = qa
retriever_for_eval = retriever
chunks_for_eval = chunks

# The evaluation functions remain the same
def evaluate_retrieval(retriever, test_cases, chunks_list):
    results = []
    for test_case in test_cases:
        retrieved = retriever.invoke(test_case["question"])
        retrieved_texts = [doc.page_content.lower() for doc in retrieved]
        keywords_found_in_chunks = [
            keyword for keyword in test_case.get("expected_chunk_keywords", [])
            if any(keyword.lower() in text for text in retrieved_texts)
        ]
        relevance_score = len(keywords_found_in_chunks) / max(len(test_case.get("expected_chunk_keywords", [""])), 1)
        results.append({
            "question": test_case["question"],
            "num_retrieved": len(retrieved),
            "keywords_found": keywords_found_in_chunks,
            "relevance_score": relevance_score,
            "retrieved_chunks": [doc.page_content[:100] + "..." for doc in retrieved]
        })
    return results

def evaluate_answers(qa_chain, test_cases):
    results = []
    for test_case in test_cases:
        try:
            answer = qa_chain.invoke({"input": test_case["question"]})
            answer_text = answer["result"].lower()
            keywords_found = [kw for kw in test_case["expected_keywords"] if kw.lower() in answer_text]
            keyword_coverage = len(keywords_found) / len(test_case["expected_keywords"]) if test_case["expected_keywords"] else 0
            answer_length = len(answer["result"])
            results.append({
                "question": test_case["question"],
                "answer": answer["result"],
                "keywords_found": keywords_found,
                "keyword_coverage": keyword_coverage,
                "answer_length": answer_length,
                "category": test_case.get("category", "unknown")
            })
        except Exception as e:
            results.append({
                "question": test_case["question"],
                "answer": f"Error: {e}",
                "keywords_found": [],
                "keyword_coverage": 0,
                "answer_length": 0,
                "category": test_case.get("category", "unknown")
            })
    return results

# Run evaluations
print("\nEvaluating Retrieval Performance...")
print("-" * 60)
retrieval_results = evaluate_retrieval(retriever_for_eval, test_cases, chunks_for_eval)

print("\nEvaluating Answer Quality...")
print("-" * 60)
answer_results = evaluate_answers(qa_for_eval, test_cases)

# Print detailed results
print("\n" + "=" * 60)
print("Detailed Evaluation Results")
print("=" * 60)

for i, (ret_result, ans_result) in enumerate(zip(retrieval_results, answer_results)):
    print(f"\nTest Case {i+1}: {ret_result['question']}")
    print(f"  Category: {ans_result.get('category', 'unknown')}")
    print(f"  Retrieved chunks: {ret_result['num_retrieved']}")
    print(f"  Retrieval relevance score: {ret_result['relevance_score']:.2%}")
    print(f"  Keywords found in chunks: {ret_result['keywords_found']}")
    print(f"  Keywords found in answer: {ans_result['keywords_found']}")
    print(f"  Keyword coverage: {ans_result['keyword_coverage']:.2%}")
    print(f"  Answer length: {ans_result['answer_length']} characters")

# Aggregate metrics
import numpy as np
avg_relevance = np.mean([r["relevance_score"] for r in retrieval_results])
avg_coverage = np.mean([r["keyword_coverage"] for r in answer_results])
avg_answer_length = np.mean([r["answer_length"] for r in answer_results])

print("\n" + "=" * 60)
print("Aggregate Metrics")
print("=" * 60)
print(f"Average retrieval relevance score: {avg_relevance:.2%}")
print(f"Average keyword coverage: {avg_coverage:.2%}")
print(f"Average answer length: {avg_answer_length:.1f} characters")
print(f"Total test cases: {len(test_cases)}")

### Build a Simple RAG Chatbot

**Task**: 
1. Create an interactive chatbot that uses the RAG system
2. Implement conversation history (optional: use LangChain's memory)
3. Add error handling and user-friendly responses

**Questions to answer**:
- How do you handle follow-up questions in a RAG system?
- Should conversation history be included in the retrieval context?
- How can you improve the user experience?


In [ ]:
def simple_rag_chatbot(qa_chain, max_turns=10, conversation_history=None):
    if conversation_history is None:
        conversation_history = []
    
    print("=" * 60)
    print("RAG Chatbot - Type 'quit' to exit, 'history' to see conversation")
    print("=" * 60)
    print("This chatbot uses RAG to answer questions based on the document knowledge base.\n")
    
    turns = 0
    while turns < max_turns:
        try:
            question = input("\nYou: ").strip()
            
            if question.lower() in ['quit', 'exit', 'bye', 'q']:
                print("\nGoodbye! Thanks for using the RAG chatbot!")
                break
            
            if question.lower() == 'history':
                print("\n" + "=" * 60)
                print("Conversation History:")
                print("=" * 60)
                for i, (q, a) in enumerate(conversation_history, 1):
                    print(f"\nTurn {i}:")
                    print(f"  You: {q}")
                    print(f"  Bot: {a[:200]}..." if len(a) > 200 else f"  Bot: {a}")
                continue
            
            if not question:
                print("Please enter a question.")
                continue
            
            print("\nBot: Thinking...")
            answer = qa_chain.invoke({"input": question})
            answer_text = answer["answer"]
            
            print(f"Bot: {answer_text}")
            conversation_history.append((question, answer_text))
            turns += 1
            
        except KeyboardInterrupt:
            print("\n\nInterrupted by user. Goodbye!")
            break
        except Exception as e:
            print(f"\nError: {e}")
            print("Please try again or type 'quit' to exit.")
    
    return conversation_history

# Demo mode: simple test questions
# "Tell me about Mr. Jonas.",
# "What happens with Jenny's toaster?",
# "Why is Noodleton famous now?"

conversation = simple_rag_chatbot(qa)

## 🎯 Summary

In this notebook, you learned:

1. **RAG Fundamentals**: How retrieval-augmented generation combines document retrieval with language model generation
2. **Pipeline Components**: Document loading, chunking, embedding, vector storage, retrieval, and generation
3. **LangChain Implementation**: How to build a RAG system using LangChain components
4. **Visualization**: How to visualize and analyze different aspects of the RAG pipeline
5. **Best Practices**: Chunk sizing, retrieval strategies, and chain types

## 🚀 Next Steps

- Explore advanced RAG techniques (reranking, query expansion, hybrid search)
- Learn about different embedding models and their trade-offs
- Experiment with different vector databases (ChromaDB, Pinecone, Weaviate)
- Implement more sophisticated retrieval strategies (BM25 + semantic search)
- Build production-ready RAG applications with error handling and monitoring

## 📚 Additional Resources

- [LangChain Documentation](https://python.langchain.com/)
- [Qdrant Documentation](https://github.com/facebookresearch/qdrant)
- [OpenAI Embeddings Guide](https://platform.openai.com/docs/guides/embeddings)
- [RAG Research Paper](https://arxiv.org/abs/2005.11401)
